Creates `data_good_sun_2020.npz` and `data_good_sun_2021.npz` by using `pvlib` to make sure the sun's elevation is at least 10 degrees.

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import pathlib
import datetime
import pvlib
import tqdm
import sys
sys.path.append('..')
import common.utils as utils


/home/sigaida/miniconda3/envs/climatehack/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# sun must have at least this elevation
MIN_ELEVATION = 10

In [2]:
SATELLITE_ZARR_PATH = "gs://public-datasets-eumetsat-solar-forecasting/satellite/EUMETSAT/SEVIRI_RSS/v3/eumetsat_seviri_hrv_uk.zarr"

dataset = xr.open_dataset(
    SATELLITE_ZARR_PATH, 
    engine="zarr",
    chunks="auto",  # Load the data as a Dask array
)

print(dataset)


<xarray.Dataset>
Dimensions:  (time: 173624, y: 891, x: 1843)
Coordinates:
  * time     (time) datetime64[ns] 2020-01-01T00:05:00 ... 2021-11-07T15:50:00
  * x        (x) float32 2.8e+04 2.7e+04 2.6e+04 ... -1.813e+06 -1.814e+06
    x_osgb   (y, x) float32 dask.array<chunksize=(891, 1843), meta=np.ndarray>
  * y        (y) float32 4.198e+06 4.199e+06 4.2e+06 ... 5.087e+06 5.088e+06
    y_osgb   (y, x) float32 dask.array<chunksize=(891, 1843), meta=np.ndarray>
Data variables:
    data     (time, y, x) int16 dask.array<chunksize=(22, 891, 1843), meta=np.ndarray>


In [7]:
# we need these to get lat/lon info, which then helps determines the sun's elevation
# this can be easiest obtained by downloading from the Google Drive linked in the repo README
# you can also just make a crop and only download the OSGB data and save that as `coords.npz`
p = pathlib.Path('coords.npz')
f = np.load(p)
x_osgb = f['x_osgb']
y_osgb = f['y_osgb']

In [3]:
# run this file twice, one for 2020 and one for 2021
YEAR = 2020
start_date = datetime.datetime(YEAR, 1, 1)
end_date = datetime.datetime(YEAR, 12, 31)

In [4]:
cur = start_date
days_to_get = []
while cur != end_date + datetime.timedelta(days=1):
    days_to_get.append(cur)
    cur = cur + datetime.timedelta(days=1)

In [5]:
len(days_to_get)

366

In [8]:
# make a lat_lon array. Index i,j gives (lat,lon)
lat_lon = np.zeros((325,400,2))
for i in range(lat_lon.shape[0]):
    for j in range(lat_lon.shape[1]):
        xo = x_osgb[i,j]
        yo = y_osgb[i,j]
        lat, lon = utils.osgb_to_lat_lon(xo, yo)
        lat_lon[i,j] = lat,lon

In [9]:
def get_day_slice(date):
    # make sure corner vertices satisfy the 10 degree elevation requirement
    points_to_check = [
        (0,0),
        (0,-1),
        (-1,0),
        (-1,-1),
    ]
    # start search from midnight to the last hour of the day
    tstart = date
    tend = date + datetime.timedelta(hours=23)
    for p in points_to_check:
        lat, lon = lat_lon[p]
        sun_is_bad = True
        while sun_is_bad:
            pd_tstart = pd.Timestamp(tstart)
            pd_tend = pd.Timestamp(tend)
            e1 = pvlib.solarposition.get_solarposition(pd_tstart, lat, lon).elevation.values[0]
            e2 = pvlib.solarposition.get_solarposition(pd_tend, lat, lon).elevation.values[0]
            sun_is_bad = False
            if e1 < MIN_ELEVATION:
                sun_is_bad = True
                tstart += datetime.timedelta(minutes=30)
            if e2 < MIN_ELEVATION:
                sun_is_bad = True
                tend -= datetime.timedelta(minutes=30)
            # not possible
            if tstart > tend:
                return None
    
    # if you don't have 3h of data, do not download
    if tend - tstart < datetime.timedelta(hours=3):
        return None
    
    data_slice = dataset.loc[
        {
            "time": slice(
                tstart,
                tend,
            )
        }
    ].isel(
        x=slice(550, 950),
        y=slice(375, 700),
    )
    
    # sometimes there is no data
    if len(data_slice.time) == 0:
        return None
    return data_slice

In [10]:
slices = []
for date in tqdm.tqdm(days_to_get):
    slc = get_day_slice(date)
    if slc is None:
        continue
    slices.append(slc)

  6%|▌         | 22/366 [00:08<02:08,  2.68it/s]


NameError: name 'dataset' is not defined

In [10]:
len(slices)

253

In [11]:
combined = xr.concat(slices, dim='time')

In [12]:
# takes a while
times = combined['time'].to_numpy()
x = combined['x'].to_numpy()
x_osgb = combined['x_osgb'].to_numpy()
y = combined['y'].to_numpy()
y_osgb = combined['y_osgb'].to_numpy()
%time data = combined['data'].to_numpy()

CPU times: user 54min 33s, sys: 1min 23s, total: 55min 56s
Wall time: 18min 47s


In [13]:
times.shape, x.shape, x_osgb.shape, y.shape, y_osgb.shape, data.shape

((30026,), (400,), (325, 400), (325,), (325, 400), (30026, 325, 400))

In [14]:
# save to data folder
p = pathlib.Path(f'data_good_sun_{YEAR}.npz')
if p.exists():
    raise ValueError(f'Path {p} already exists!')

np.savez(
    p,
    times=times,
    x=x,
    x_osgb=x_osgb,
    y=y,
    y_osgb=y_osgb,
    data=data,
)